# Determining the Ideal "g" Function

The $g$ function is used to weight the importance of points at varying distances from the isocenter as we optimize $f$.


$$
f(x, y, z, \theta, \phi, \xi) =
\sum_B g(\left|\mathbf{b}\right|)
\;
(\left|\mathbf{b} - \mathbf{a}_{\textrm{S},\textrm{min}}\right|/\rho(|\mathbf{b}|) - 1)
\;
u(\rho(|\mathbf{b}|) - \left|\mathbf{b} - \mathbf{a}_{\textrm{S},\textrm{min}}\right|)
$$

This formulation assumes that $g$ is a function of the radial distance from the isocenter (i.e. it is spherically symmetric).  This seems most natural given that the error tends to increase with distance from the isocen, however given that the MRI machine is not spherically symmetric, it may also be worth considering shapes for $g$ that are no spherically symmetric.

It is not clear what the proper form for $g$ should be.

The simplest shape for $g$ would be a sharp cutoff beyond some distance.  This shape would weight all points within the cutoff equally.

Probably a better shape for $g$ would be a montonically decreasing function that is derived from the expected error (and thus expected confidence) of the points as a function of their distance from the isocenter.

The rest of this notebook investigates the simple form of $g$--namely a $g$ which is a 1 for points below a threshold distance and 0 otherwise.  For this function, the main question is what should the cutoff be?

Low cutoff distances will ensure distorted points do not contribute to the registration.

High cutoff distances may be more stable, and errors and noise in the central points will be less able to cause bad rotatations and scaling; this is because points further from the center will counter balance and averge out errors that may be in the points in the center.

In [ ]:
%matplotlib inline

from scipy.io import loadmat
import scipy
import numpy as np
import matplotlib.pyplot as plt

import visualization

points_1 = loadmat('../data/points/mircea_points_1.mat')['points']
points_2 = loadmat('../data/points/mircea_points_2.mat')['points']

## Here we plot the two sets of points

In [ ]:
fig = visualization.scatter3({'points_1': points_1, 'points_2': points_2})

In [ ]:
from registration import register
import affine

def transform(A, result):
    transformation_matrix = affine.translation_rotation(*result)
    return affine.apply_affine(transformation_matrix, A)

def results_str(result):
    return 'x={:+.4f}, y={:+.4f}, z={:+.4f}, theta={:+.4f}, phi={:+.4f}, xi={:+.4f}'.format(*result)

def register_shift_scatter(A, B, g, rho):
    result = register(A, B, g, rho, 1e-8)
    A_shifted = transform(A, result)
    visualization.scatter3({'A_shifted': A_shifted, 'B': B})
    plt.title(results_str(result))

## Here we plot the registered points, where a cutoff of 50 mm was used in $g$

In [ ]:
g_cutoff_50 = lambda mag_b: 0 if mag_b > 50 else 1
rho = lambda mag_b: 10
register_shift_scatter(points_1, points_2, g_cutoff_15, rho)

## Here we investigate how the cutoff affects the resulting optimzation result

Note that the "number of points" matched is slightly approximated, because it is only based on the position of the points in $B$, and not on the positions of the points in $A$.

In [ ]:
from registration import register
from scipy.linalg import norm
import affine

def contained_points(g, points):
    return len(list(filter(lambda p: g(norm(p)) > 0, points.T)))

# Iterate through various extents for g, printing out the results as we go
g_cutoffs = range(10, 80, 5)
for cutoff in g_cutoffs:
    g = lambda mag_b: 0 if mag_b > cutoff else 1
    rho = lambda mag_b: 10
    result = register(points_1, points_2, g, rho, 1e-8)
    matched_B = contained_points(g, points_1)
    print('Using radius of {:3d}, matched {:3d} points: {}'.format(cutoff, matched_B, results_str(result)))

# Conclusion

It seems that, at least with this data set, the number of points does not substantially affect the registration.

With very small numbers of points (e.g. 20), there is some error due to the noise in the central points, however this averages out quickly.

That said, it appears that there is random noise added to the entire set of points.  It will be interesting to consider how the choice of $g$ affects the registration process in more realistic scenarios.  In particular, how does the shape of $g$ (and $\rho$ for that matter) affect the registration when:

- the geometric distortion increases as we move away from the origin
- there are false positives on the periphery, and in the inside of the dataset
- there are false negatives (especially interesting is when there are false negatives in the middle set of points)
- there are non-random, systematic distortions on the periphery